# 第2章 商品推薦を実現する数理

このNotebookでは、レコメンドシステムを実現するための数理的基礎を学びます。

**キーワード**：協調フィルタリング、行列因子分解、最小二乗法、勾配降下法、評価値行列、コサイン類似度、指示関数、損失関数、内積、三角関数、ベクトル、微分、偏微分、行列

## 目次
- 2-1 はじめに
- 2-2 商品の評価を数理的に表現する ー評価値行列ー
- 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー
- 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー
- 2-5 コサイン類似度の意味を考える ー三角関数ー
- 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー
- 2-7 コサイン類似度を改良する ー中心化ー
- 2-8 コサイン類似度を計算する ー指示関数ー
- 2-9 欠損値を推計する数理モデルを設計し、計算を実行する
- 2-10 アイテム同士の類似度で予測値を推計する
- 2-11 ユーザー目線で数理モデルを再考する ーセレンディピティー
- 2-12 課題解決のために数理モデルを変更する ー行列因子分解ー
- 2-13 評価値の推計を最適化問題に置き換える ー残差行列と誤差ー

## 環境設定

In [ ]:
# 日本語フォントのインストール（Google Colab用）
!pip install japanize-matplotlib -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの設定
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

---
# 2-1 はじめに

私たちの生活は**レコメンド**に溢れています。

- オンラインショッピングの「おすすめの商品」
- YouTubeの「あなたへのおすすめ動画」
- SNSの「興味のある投稿」

レコメンドは、レコメンドのために作られた**数理モデル**が膨大なデータを処理することで実現します。

本章では、レコメンドを実現する基礎的な数学的手法に焦点を当てて解説します。具体的には、**協調フィルタリング**と**行列因子分解**と呼ばれる数理的手法を中心に解説し、いかにして数理モデルが私たちの「好み」を理解し、購入する可能性が高い商品やコンテンツをレコメンドしているのかを解き明かします。

---
# 2-2 商品の評価を数理的に表現する ー評価値行列ー

レコメンドを数理モデルで実現するために、まず**評価値行列**を作成します。

## ベクトルによる表現

ベクトルは数値の順序付きリストです。

$$\vec{r} = \begin{pmatrix} r_1 \\ r_2 \\ \vdots \\ r_n \end{pmatrix}, \quad \vec{r}^T = (r_1, r_2, \cdots, r_n)$$

- 左側の式は縦の「列方向」に数値が並んでいるので**列ベクトル**
- 右側の式は横の「行方向」に数値が並んでいるので**行ベクトル**
- $^T$ は**転置**を表す記号で、行と列の方向を変える操作を指す

In [ ]:
# ユーザーごとの評価値をベクトルで表現
# 図2.3の例：4人のユーザーが4つのアイテムを評価
# 0は欠損値（未評価）を表す

# 行ベクトル表現
R_u1 = np.array([2, 3, 0, 5])  # user1の評価
R_u2 = np.array([2, 5, 0, 5])  # user2の評価
R_u3 = np.array([0, 3, 4, 4])  # user3の評価
R_u4 = np.array([4, 2, 3, 0])  # user4の評価

print("=== ユーザーごとの評価ベクトル（行ベクトル） ===")
print(f"R_u1 = {R_u1}")
print(f"R_u2 = {R_u2}")
print(f"R_u3 = {R_u3}")
print(f"R_u4 = {R_u4}")

In [ ]:
# 列ベクトル表現（アイテムごとの評価）
R_i1 = np.array([[2], [2], [0], [4]])  # item1の評価
R_i2 = np.array([[3], [5], [3], [2]])  # item2の評価
R_i3 = np.array([[0], [0], [4], [3]])  # item3の評価
R_i4 = np.array([[5], [5], [4], [0]])  # item4の評価

print("=== アイテムごとの評価ベクトル（列ベクトル） ===")
print(f"R_i1 = {R_i1.flatten()}")
print(f"R_i2 = {R_i2.flatten()}")
print(f"R_i3 = {R_i3.flatten()}")
print(f"R_i4 = {R_i4.flatten()}")

## 評価値行列 R

行ベクトル $R_{u_1} \sim R_{u_4}$ を列方向（縦）に並べると、評価値を1つにまとめた**行列**で表現できます。これを**評価値行列**と呼び、$R$と表記します。

$$R = \begin{pmatrix} R_{u_1} \\ R_{u_2} \\ R_{u_3} \\ R_{u_4} \end{pmatrix} = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

- $R$ は Rating（評価）の頭文字
- 0 は未評価、つまり**欠損値**を示す
- user1のitem1に対する評価値を $r_{1,1}$ と表記する

一般化すると、ユーザーの総数を $U$、アイテムの総数を $I$ とすると：

$$R = \begin{pmatrix} r_{1,1} & r_{1,2} & \cdots & r_{1,I} \\ r_{2,1} & r_{2,2} & \cdots & r_{2,I} \\ \vdots & \vdots & \ddots & \vdots \\ r_{U,1} & r_{U,2} & \cdots & r_{U,I} \end{pmatrix}$$

In [ ]:
# 評価値行列の作成
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("=== 評価値行列 R ===")
print(R)
print(f"\n行列のサイズ: {R.shape[0]}ユーザー × {R.shape[1]}アイテム")

In [ ]:
# 評価値行列の可視化
fig, ax = plt.subplots(figsize=(8, 6))

# ヒートマップを作成（0は欠損値として特別扱い）
masked_R = np.ma.masked_where(R == 0, R)
im = ax.imshow(masked_R, cmap='YlOrRd', vmin=1, vmax=5)

# 欠損値（0）をグレーで表示
ax.imshow(np.where(R == 0, 1, np.nan), cmap='gray', vmin=0, vmax=1, alpha=0.3)

# 軸ラベル
ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 各セルに値を表示
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            text = ax.text(j, i, '−', ha='center', va='center', color='gray', fontsize=14)
        else:
            text = ax.text(j, i, R[i, j], ha='center', va='center', color='black', fontsize=14)

ax.set_title('評価値行列 R（グレー: 欠損値）')
plt.colorbar(im, label='評価値')
plt.tight_layout()
plt.show()

---
# 2-3 評価値の予測を数理モデルで実現する ー協調フィルタリングと行列因子分解ー

## レコメンドの前提条件

- ユーザーは、購入したすべての商品に評価値を付与している
- 逆に、ユーザーは購入していない商品に評価値を付与していない
- レコメンドする商品は、そのユーザーが未購入の商品（評価値が欠損している商品）に限る

つまり、**欠損部分の評価値を予測値として算出できれば、「もし購入したらいくらの評価を付けるか」を推定できる**。これがレコメンドの有効な判断指標となります。

In [ ]:
# レコメンドの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：評価値がある部分とない部分
ax1 = axes[0]
colors = np.where(R == 0, 0.3, 0.8)
ax1.imshow(colors, cmap='Blues', vmin=0, vmax=1)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax1.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
        else:
            ax1.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)
ax1.set_xticks(range(4))
ax1.set_yticks(range(4))
ax1.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax1.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax1.set_title('評価値行列（? = 欠損値 = レコメンド対象）')

# 右：予測値の例
ax2 = axes[1]
R_predicted = R.copy().astype(float)
R_predicted[0, 2] = 4.2  # user1のitem3を予測
R_predicted[1, 2] = 4.5  # user2のitem3を予測
R_predicted[2, 0] = 2.1  # user3のitem1を予測
R_predicted[3, 3] = 3.8  # user4のitem4を予測

im2 = ax2.imshow(R_predicted, cmap='YlOrRd', vmin=1, vmax=5)
for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            ax2.text(j, i, f'{R_predicted[i,j]:.1f}', ha='center', va='center', 
                    fontsize=12, color='blue', fontweight='bold')
        else:
            ax2.text(j, i, str(int(R_predicted[i, j])), ha='center', va='center', fontsize=14)
ax2.set_xticks(range(4))
ax2.set_yticks(range(4))
ax2.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax2.set_yticklabels(['user1', 'user2', 'user3', 'user4'])
ax2.set_title('予測後（青字 = 予測値）')
plt.colorbar(im2, ax=ax2, label='評価値')

plt.tight_layout()
plt.show()

print("例：user1のitem3の予測値が4.2なら、item3をuser1にレコメンドすべき！")

## 予測値の推計方法：3つのアプローチ

| 手法 | 説明 | 軸 |
|:---|:---|:---|
| ①userを軸とした予測値の算出 | 各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出 | user |
| ②itemを軸とした予測値の算出 | 各userの評価が似ているitemを選び、そのitemの評価値を用いて予測値を算出 | item |
| ③行列因子分解（MF） | 評価値行列を分解した潜在因子行列を解析的に求めることで、予測値を算出 | − |

①と②は**協調フィルタリング**という手法に分類されます。
③は協調フィルタリングとは異なる手法で、評価値行列を**潜在因子行列**と呼ばれる行列に分解して欠損値を推定するアプローチです。

In [ ]:
# 3つのアプローチの概念図
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# ①userを軸とした予測
ax1 = axes[0]
ax1.text(0.5, 0.9, '①userを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax1.transAxes)
ax1.text(0.5, 0.7, '似ているユーザーの評価値を利用', ha='center', fontsize=10, transform=ax1.transAxes)
ax1.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax1.text(0.2, 0.4, 'user1', fontsize=10, ha='center', va='center')
ax1.text(0.8, 0.4, 'user2\n(類似)', fontsize=10, ha='center', va='center', color='blue')
ax1.text(0.5, 0.2, '→ user2の評価を参考にuser1の欠損を予測', ha='center', fontsize=9, transform=ax1.transAxes)
ax1.axis('off')

# ②itemを軸とした予測
ax2 = axes[1]
ax2.text(0.5, 0.9, '②itemを軸とした予測値の算出', ha='center', fontsize=11, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.5, 0.7, '似ているアイテムの評価値を利用', ha='center', fontsize=10, transform=ax2.transAxes)
ax2.annotate('', xy=(0.7, 0.4), xytext=(0.3, 0.4),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax2.text(0.2, 0.4, 'item1', fontsize=10, ha='center', va='center')
ax2.text(0.8, 0.4, 'item2\n(類似)', fontsize=10, ha='center', va='center', color='green')
ax2.text(0.5, 0.2, '→ item2の評価を参考にitem1の欠損を予測', ha='center', fontsize=9, transform=ax2.transAxes)
ax2.axis('off')

# ③行列因子分解
ax3 = axes[2]
ax3.text(0.5, 0.9, '③行列因子分解（MF）', ha='center', fontsize=11, fontweight='bold', transform=ax3.transAxes)
ax3.text(0.5, 0.7, 'R ≈ P × Q', ha='center', fontsize=12, transform=ax3.transAxes)
ax3.text(0.15, 0.4, 'R', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax3.text(0.32, 0.4, '≈', fontsize=14, ha='center')
ax3.text(0.5, 0.4, 'P', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax3.text(0.62, 0.4, '×', fontsize=14, ha='center')
ax3.text(0.75, 0.4, 'Q', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow'))
ax3.text(0.5, 0.2, '→ 潜在因子行列に分解して欠損を推定', ha='center', fontsize=9, transform=ax3.transAxes)
ax3.axis('off')

plt.tight_layout()
plt.show()

---
# 2-4 ユーザー同士の類似度で予測値を推計する ー内積の定理とコサイン類似度ー

まずは「①userを軸とした予測値の算出」について解説します。

この手法では、各itemの評価が似ているuserを選び、そのuserの評価値を用いて予測値を算出します。
このとき、「各itemの評価が似ているuserを選ぶ」という処理を実現するために、**コサイン類似度**という手法を採用します。

## 基礎となる数理的手法と情報工学的アプローチ

| 基礎となる数理的手法 | 情報工学的アプローチ |
|:---|:---|
| 内積の定理：$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta$ | コサイン類似度：$\cos(a, b) = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$ |
| ベクトルの大きさ（三平方の定理）：$|\vec{a}| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2}$ | |

## 座標空間での表現

例として、userが6人、itemが2個だけの状況を想定し、以下の評価値行列が作成されたとしましょう。

$$R = \begin{pmatrix} 2 & 3 \\ 2 & 5 \\ 0 & 3 \\ 4 & 2 \\ 4 & 4 \\ 3 & 0 \end{pmatrix}$$

この評価値行列を**座標空間**で示すと、ユーザー同士の類似度合いが視覚的に把握できます。

In [ ]:
# 6人のユーザー、2つのアイテム
R_simple = np.array([
    [2, 3],  # user1
    [2, 5],  # user2
    [0, 3],  # user3 (item1未評価)
    [4, 2],  # user4
    [4, 4],  # user5
    [3, 0]   # user6 (item2未評価)
])

print("評価値行列 R:")
print(R_simple)

In [ ]:
# 図2.10: 座標空間でのプロット
fig, ax = plt.subplots(figsize=(8, 8))

users = ['user1', 'user2', 'user3', 'user4', 'user5', 'user6']
colors = ['blue', 'blue', 'gray', 'blue', 'blue', 'gray']

for i, (user, color) in enumerate(zip(users, colors)):
    x, y = R_simple[i]
    if x == 0 or y == 0:
        ax.scatter(x, y, s=100, c='gray', alpha=0.5, zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10, color='gray')
    else:
        ax.scatter(x, y, s=100, c='blue', zorder=5)
        ax.annotate(user, (x, y), xytext=(5, 5), textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1への評価値', fontsize=12)
ax.set_ylabel('item2への評価値', fontsize=12)
ax.set_title('図2.10: 座標空間でのユーザー分布', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax.text(-0.3, 0, '評価\nなし', fontsize=9, va='center', color='gray')
ax.text(0, -0.3, '評価なし', fontsize=9, ha='center', color='gray')

plt.tight_layout()
plt.show()

## ベクトルによる表現

ここで、目的を「user1に似ているユーザーが誰かを特定すること」とし、コサイン類似度によって類似ユーザーを特定します。

考えやすいように、考察対象をuser1, user2, user4に絞り、各userをベクトルで表現します。

$$\vec{u}_1 = (2, 3), \quad \vec{u}_2 = (2, 5), \quad \vec{u}_4 = (4, 2)$$

In [ ]:
# 図2.11: ベクトルとしての表現
fig, ax = plt.subplots(figsize=(8, 8))

u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

ax.quiver(0, 0, u1[0], u1[1], angles='xy', scale_units='xy', scale=1, 
          color='blue', width=0.02, label=f'$\\vec{{u}}_1$ = (2, 3)')
ax.quiver(0, 0, u2[0], u2[1], angles='xy', scale_units='xy', scale=1, 
          color='green', width=0.02, label=f'$\\vec{{u}}_2$ = (2, 5)')
ax.quiver(0, 0, u4[0], u4[1], angles='xy', scale_units='xy', scale=1, 
          color='red', width=0.02, label=f'$\\vec{{u}}_4$ = (4, 2)')

ax.plot(u1[0], u1[1], 'bo', markersize=8)
ax.plot(u2[0], u2[1], 'go', markersize=8)
ax.plot(u4[0], u4[1], 'ro', markersize=8)

ax.annotate(f'({u1[0]},{u1[1]})\nuser1', (u1[0], u1[1]), xytext=(10, 5), textcoords='offset points', fontsize=10)
ax.annotate(f'({u2[0]},{u2[1]})\nuser2', (u2[0], u2[1]), xytext=(10, 5), textcoords='offset points', fontsize=10)
ax.annotate(f'({u4[0]},{u4[1]})\nuser4', (u4[0], u4[1]), xytext=(10, -15), textcoords='offset points', fontsize=10)

ax.set_xlim(-0.5, 5.5)
ax.set_ylim(-0.5, 5.5)
ax.set_xlabel('item1', fontsize=12)
ax.set_ylabel('item2', fontsize=12)
ax.set_title('図2.11: user1, user2, user4をベクトルで表現', fontsize=14)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

print("図から、user1に似ているのはuser2だと言えそうです。")
print("なぜなら、ベクトルを示す矢印のuser1とuser2が作る角度がuser1とuser4が作る角度よりも小さいからです。")

## 内積の定理

角度を「目」で見て類似度を評価することはできますが、コンピュータは「目」で判断することはできず、**数値**でしか判断できません。

そこで、この角度を**定量化**します。このときに便利な数理的手法が高校数学で学習した**内積の定理**です。

2つのベクトル $\vec{a}$, $\vec{b}$ があり、$\vec{a}$, $\vec{b}$ のなす角を $\theta$（theta：シータ）とします。

すると、**内積**は式 (2-3) のように定義されます。

$$\vec{a} \cdot \vec{b} = |\vec{a}||\vec{b}|\cos\theta \quad (2\text{-}3)$$

## cos θ の計算

$$\cos\theta = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \quad (2\text{-}4)$$

### 分子：内積 $\vec{a} \cdot \vec{b}$ の計算

$$\vec{a} \cdot \vec{b} = a_1 b_1 + a_2 b_2 \quad (2\text{-}6)$$

### 分母：ベクトルの大きさ（ノルム）の計算

$$|\vec{a}| = \sqrt{a_1^2 + a_2^2} \quad (2\text{-}10)$$

In [ ]:
# コサイン類似度の計算（基本版）
def cosine_similarity(a, b):
    """
    2つのベクトルのコサイン類似度を計算する
    式: cos(a, b) = (a · b) / (|a| × |b|)
    """
    a = np.array(a)
    b = np.array(b)
    
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

print("cosine_similarity関数を定義しました")

In [ ]:
# user1, user2, user4のコサイン類似度を計算
u1 = np.array([2, 3])
u2 = np.array([2, 5])
u4 = np.array([4, 2])

print("=== コサイン類似度の計算 ===")
print(f"u1 = {tuple(u1)}")
print(f"u2 = {tuple(u2)}")
print(f"u4 = {tuple(u4)}")
print()

cos_12 = cosine_similarity(u1, u2)
cos_14 = cosine_similarity(u1, u4)

print(f"cos(u1, u2) = {cos_12:.4f}")
print(f"cos(u1, u4) = {cos_14:.4f}")
print()
print(f"結論: cos(u1, u2) = {cos_12:.3f} > cos(u1, u4) = {cos_14:.3f}")
print("→ user2のほうがuser1に似ている")

---
# 2-5 コサイン類似度の意味を考える ー三角関数ー

## 単位円を用いた cos θ の定義

**単位円**とは原点を中心とした半径1の円です。

- 単位円上に点 $P$ を取り、原点 $O$ を中心に $x$ 軸の正の部分から反時計回りに $\theta$ だけ回転させた線分 $OP$ について
- $P$ の $x$ 座標を $\cos\theta$ と定義します
- $P$ の $y$ 座標は $\sin\theta$ と定義します

## θ と cos θ の関係

$0 \leq \theta \leq 180°$ のとき、$\theta$ の値が大きくなるほど $\cos\theta$ の値が小さくなります。

$$-1 \leq \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} \leq 1 \quad (2\text{-}7)$$

だから、計算結果が**1に近いほど**、2つのベクトルは「**似ている**」と言えるのです。

In [ ]:
# θ と cos θ の関係グラフ
fig, ax = plt.subplots(figsize=(10, 5))

theta_deg = np.linspace(0, 180, 100)
theta_rad = np.radians(theta_deg)
cos_values = np.cos(theta_rad)

ax.plot(theta_deg, cos_values, 'b-', linewidth=2)

important_points = [(0, 1, '同じ方向\n(最も類似)'), 
                    (90, 0, '直交\n(無関係)'), 
                    (180, -1, '反対方向\n(最も非類似)')]

for angle, cos_val, label in important_points:
    ax.plot(angle, cos_val, 'ro', markersize=10)
    ax.annotate(f'{label}\n({angle}°, {cos_val})', xy=(angle, cos_val),
                xytext=(angle+15, cos_val), fontsize=10,
                arrowprops=dict(arrowstyle='->', color='gray'))

ax.axhline(y=0, color='k', linewidth=0.5, linestyle='--')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values > 0), alpha=0.3, color='green', label='類似 (cos θ > 0)')
ax.fill_between(theta_deg, 0, cos_values, where=(cos_values < 0), alpha=0.3, color='red', label='非類似 (cos θ < 0)')

ax.set_xlabel('角度 θ (度)', fontsize=12)
ax.set_ylabel('cos θ', fontsize=12)
ax.set_title('角度θとcos θの関係', fontsize=14)
ax.set_xticks([0, 30, 60, 90, 120, 150, 180])
ax.set_ylim(-1.2, 1.2)
ax.grid(True, alpha=0.3)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

---
# 2-6 コサイン類似度を複数のアイテムに適用する ー多次元への拡張ー

## n次元への拡張

アイテム数が $n$ 個になった場合：

$$\vec{a} \cdot \vec{b} = \sum_{k=1}^{n} a_k b_k \quad (2\text{-}9)$$

$$|\vec{a}| = \sqrt{\sum_{k=1}^{n} a_k^2} \quad (2\text{-}12)$$

## 一般化されたコサイン類似度の公式

$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}} \quad (2\text{-}13)$$

---
# 2-7 コサイン類似度を改良する ー中心化ー

## 問題点：評価が「逆」のユーザー

例えば、item1〜5について評価が「逆」になっている2人のuserの評価値ベクトルを考えます。

$$\vec{u}_A = (1, 2, 3, 4, 5), \quad \vec{u}_B = (5, 4, 3, 2, 1)$$

両者の評価は真逆なので、コサイン類似度は低く算出されて欲しいところです。

In [ ]:
# 評価が「逆」のユーザーでコサイン類似度を計算
u_A = np.array([1, 2, 3, 4, 5])
u_B = np.array([5, 4, 3, 2, 1])

print("=== 評価が「逆」のユーザー ===")
print(f"u_A = {u_A}")
print(f"u_B = {u_B}")
print()

# 内積の計算
dot_AB = np.dot(u_A, u_B)
print(f"内積: u_A · u_B = 1×5 + 2×4 + 3×3 + 4×2 + 5×1 = {dot_AB}")

# ノルムの計算
norm_A = np.linalg.norm(u_A)
norm_B = np.linalg.norm(u_B)
print(f"|u_A| = √(1² + 2² + 3² + 4² + 5²) = √{sum(u_A**2)} = {norm_A:.4f}")
print(f"|u_B| = √(5² + 4² + 3² + 2² + 1²) = √{sum(u_B**2)} = {norm_B:.4f}")

# コサイン類似度
cos_AB = cosine_similarity(u_A, u_B)
print(f"\ncos(u_A, u_B) = {dot_AB} / ({norm_A:.4f} × {norm_B:.4f}) = {cos_AB:.3f}")
print()
print("問題：評価が真逆なのに、コサイン類似度が0.636と高い！")
print("これは最大値1に近く、「似ている」と判断されてしまう。")

## 解決策：中心化（Centering）

この評価値の差を緩和します。具体的には、userごとに各アイテムへの評価値の**平均値**を算出し、その平均値を各評価値から引きます。

この処理を**中心化**と言います。

In [ ]:
# 図2.21: 中心化の可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

items = ['item1', 'item2', 'item3', 'item4', 'item5']

# 左：中心化前
ax1 = axes[0]
x = np.arange(len(items))
width = 0.35
ax1.bar(x - width/2, u_A, width, label='user A', color='blue', alpha=0.7)
ax1.bar(x + width/2, u_B, width, label='user B', color='orange', alpha=0.7)
ax1.set_xticks(x)
ax1.set_xticklabels(items)
ax1.set_ylabel('評価値')
ax1.set_title('中心化前')
ax1.legend()
ax1.axhline(y=np.mean(u_A), color='blue', linestyle='--', alpha=0.5, label=f'A平均={np.mean(u_A)}')
ax1.axhline(y=np.mean(u_B), color='orange', linestyle='--', alpha=0.5, label=f'B平均={np.mean(u_B)}')
ax1.set_ylim(0, 6)

# 中心化
u_A_centered = u_A - np.mean(u_A)
u_B_centered = u_B - np.mean(u_B)

# 右：中心化後
ax2 = axes[1]
ax2.bar(x - width/2, u_A_centered, width, label='user A (中心化後)', color='blue', alpha=0.7)
ax2.bar(x + width/2, u_B_centered, width, label='user B (中心化後)', color='orange', alpha=0.7)
ax2.set_xticks(x)
ax2.set_xticklabels(items)
ax2.set_ylabel('評価値（中心化後）')
ax2.set_title('中心化後')
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.set_ylim(-3, 3)

plt.suptitle('図2.21: ユーザーごとに評価値の平均値を算出し、各評価値から減算する', fontsize=12)
plt.tight_layout()
plt.show()

print(f"u_A 平均値: {np.mean(u_A)}")
print(f"u_A 中心化後: {u_A_centered}")
print(f"u_B 平均値: {np.mean(u_B)}")
print(f"u_B 中心化後: {u_B_centered}")

In [ ]:
# 中心化後のコサイン類似度を計算
print("=== 中心化後のコサイン類似度計算 ===")
print(f"u_A (中心化後) = {u_A_centered}")
print(f"u_B (中心化後) = {u_B_centered}")
print()

# 内積
dot_AB_centered = np.dot(u_A_centered, u_B_centered)
print(f"内積: {u_A_centered[0]:.0f}×{u_B_centered[0]:.0f} + {u_A_centered[1]:.0f}×{u_B_centered[1]:.0f} + ... = {dot_AB_centered}")

# ノルム
norm_A_centered = np.linalg.norm(u_A_centered)
norm_B_centered = np.linalg.norm(u_B_centered)
print(f"|u_A| = √{sum(u_A_centered**2):.0f} = {norm_A_centered:.4f}")
print(f"|u_B| = √{sum(u_B_centered**2):.0f} = {norm_B_centered:.4f}")

# コサイン類似度
cos_AB_centered = cosine_similarity(u_A_centered, u_B_centered)
print(f"\ncos(u_A, u_B) = {dot_AB_centered} / ({norm_A_centered:.4f} × {norm_B_centered:.4f}) = {cos_AB_centered:.3f}")
print()
print("結果：コサイン類似度が -1 になりました！")
print("これは最小値で、「評価が真逆」という判断と一致します。")

## 改良したコサイン類似度の公式

各要素から平均値を引いた計算が実行されればよいので、改良したモデルは式 (2-14) のようになります。

$\bar{a}$, $\bar{b}$ は、それぞれ $\vec{a}$, $\vec{b}$ の平均評価値を表しています。

$$\cos(a, b) = \frac{\sum_{k=1}^{n} (a_k - \bar{a})(b_k - \bar{b})}{\sqrt{\sum_{k=1}^{n} (a_k - \bar{a})^2} \sqrt{\sum_{k=1}^{n} (b_k - \bar{b})^2}} \quad (2\text{-}14)$$

In [ ]:
# 中心化付きコサイン類似度関数
def cosine_similarity_centered(a, b, mask_a=None, mask_b=None):
    """
    中心化を行ったコサイン類似度を計算する
    
    Parameters:
    -----------
    a, b : array-like
        評価ベクトル
    mask_a, mask_b : array-like, optional
        評価が存在する要素を示すマスク（Trueの要素のみ平均計算に使用）
    
    Returns:
    --------
    float
        中心化されたコサイン類似度
    """
    a = np.array(a, dtype=float)
    b = np.array(b, dtype=float)
    
    # マスクが指定されていない場合は、0以外を有効とする
    if mask_a is None:
        mask_a = (a != 0)
    if mask_b is None:
        mask_b = (b != 0)
    
    # 共通して評価しているアイテムのみ使用
    common_mask = mask_a & mask_b
    
    if np.sum(common_mask) == 0:
        return 0.0
    
    # 平均値を計算（共通アイテムのみ）
    mean_a = np.mean(a[common_mask])
    mean_b = np.mean(b[common_mask])
    
    # 中心化
    a_centered = a[common_mask] - mean_a
    b_centered = b[common_mask] - mean_b
    
    # コサイン類似度を計算
    dot_product = np.dot(a_centered, b_centered)
    norm_a = np.linalg.norm(a_centered)
    norm_b = np.linalg.norm(b_centered)
    
    if norm_a == 0 or norm_b == 0:
        return 0.0
    
    return dot_product / (norm_a * norm_b)

print("cosine_similarity_centered関数を定義しました")

---
# 2-8 コサイン類似度を計算する ー指示関数ー

## 指示関数 $\delta_{u,i}$

「評価値が与えられている要素のみを計算対象とする」という制約を与えています。しかし、コサイン類似度に用いられている $\sum$ は、都合よく「これは足す」「これは足さない」という処理をしてくれるわけではありません。

このような条件2の操作を実現するために、**指示関数** $\delta_{u,i}$ を式 (2-15) のように定義します。$u$ はユーザーを示す行、$i$ はアイテムを示す列を示しています。

$$\delta_{u,i} = \begin{cases} 1 & (r_{u,i}\text{が評価値行列に含まれている}) \\ 0 & (r_{u,i}\text{が評価値行列で欠損している}) \end{cases} \quad (2\text{-}15)$$

## 指示関数を組み込んだコサイン類似度

$$\cos(u_1, u_2) = \frac{\sum_{i=1}^{4} \delta_{1,i}(r_{1,i} - \bar{u}_1) \delta_{2,i}(r_{2,i} - \bar{u}_2)}{\sqrt{\sum_{i=1}^{4} \delta_{1,i}(r_{1,i} - \bar{u}_1)^2} \sqrt{\sum_{i=1}^{4} \delta_{2,i}(r_{2,i} - \bar{u}_2)^2}}$$

これにより、欠損している部分は計算から除外され、欠損していない要素のみに対して計算が実行されます。

In [ ]:
# 評価値行列R（図2.24）
R = np.array([
    [2, 3, 0, 5],  # user1: item3が欠損
    [2, 5, 0, 5],  # user2: item3が欠損
    [0, 3, 4, 4],  # user3: item1が欠損
    [4, 2, 3, 0]   # user4: item4が欠損
])

print("=== 評価値行列 R ===")
print(R)
print("\n（0は欠損値を表す）")

In [ ]:
# user1とuser2のコサイン類似度を計算（中心化・指示関数付き）
print("=== cos(u1, u2) の計算 ===")
print()

u1 = R[0]  # [2, 3, 0, 5]
u2 = R[1]  # [2, 5, 0, 5]

print(f"user1: {u1}")
print(f"user2: {u2}")
print()

# 条件1: 評価値を付けていないitemは対象外として平均値を計算
# user1とuser2共通で評価しているのはitem1, item2, item4
common_items = (u1 != 0) & (u2 != 0)
print(f"共通して評価しているアイテム: {['item1', 'item2', 'item3', 'item4'][i] for i, v in enumerate(common_items) if v}")
print(f"→ item1, item2, item4 (item3は両者とも欠損)")
print()

# 平均値の計算（共通アイテムのみ）
mean_u1 = np.mean(u1[common_items])
mean_u2 = np.mean(u2[common_items])
print(f"user1の平均評価値: (2 + 3 + 5) / 3 = {mean_u1:.3f}")
print(f"user2の平均評価値: (2 + 5 + 5) / 3 = {mean_u2:.3f}")
print()

# 中心化
u1_centered = u1[common_items] - mean_u1
u2_centered = u2[common_items] - mean_u2
print(f"user1 (中心化後): {u1_centered}")
print(f"user2 (中心化後): {u2_centered}")
print()

# コサイン類似度の計算
cos_u1_u2 = cosine_similarity_centered(u1, u2)
print(f"cos(u1, u2) = {cos_u1_u2:.3f}")

In [ ]:
# 全ユーザー間のコサイン類似度行列を計算（図2.26）
print("=== 図2.26: ユーザー間の類似度の計算結果を表形式で整理 ===")
print()

n_users = R.shape[0]
similarity_matrix = np.zeros((n_users, n_users))

for i in range(n_users):
    for j in range(n_users):
        similarity_matrix[i, j] = cosine_similarity_centered(R[i], R[j])

# 表形式で表示
print("          user1    user2    user3    user4")
for i, row in enumerate(similarity_matrix):
    print(f"user{i+1}  ", end="")
    for val in row:
        print(f"{val:8.3f} ", end="")
    print()

In [ ]:
# 類似度行列のヒートマップ
fig, ax = plt.subplots(figsize=(8, 6))

im = ax.imshow(similarity_matrix, cmap='RdYlGn', vmin=-1, vmax=1)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['user1', 'user2', 'user3', 'user4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

for i in range(4):
    for j in range(4):
        ax.text(j, i, f'{similarity_matrix[i, j]:.3f}', ha='center', va='center', fontsize=11)

ax.set_title('図2.26: ユーザー間のコサイン類似度行列')
plt.colorbar(im, label='コサイン類似度')
plt.tight_layout()
plt.show()

print("対角線は自分自身との類似度なので1.000")
print("user3とuser4の類似度が最も高い（0.894）")

---
# 2-9 欠損値を推計する数理モデルを設計し、計算を実行する

ここまでの計算結果を用いて、これから欠損値の推計を行っていきます。

## 推計対象

推計の対象となる欠損値を $r_{3,1}$（user3のitem1への評価）とします。

図2.26の表の計算結果を踏まえ、user3に似ている順番に他のユーザーを並べると
$$\cos(u_3, u_4) > \cos(u_3, u_1) > \cos(u_3, u_2)$$
となるので、user3に似ているのはuser4、user1ということにしましょう。

In [ ]:
# 図2.27: 欠損値推計の概念図
fig, ax = plt.subplots(figsize=(10, 6))

# 評価値行列を描画
ax.imshow(np.where(R == 0, 0.3, 0.8), cmap='Blues', vmin=0, vmax=1)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            if i == 2 and j == 0:  # user3のitem1（推計対象）
                ax.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
            else:
                ax.text(j, i, '−', ha='center', va='center', fontsize=14, color='gray')
        else:
            ax.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 矢印で類似ユーザーからの参照を示す
ax.annotate('', xy=(0, 2), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.annotate('', xy=(0, 2), xytext=(0, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.text(4.5, 0, 'user1\ncos=0.614', fontsize=10, va='center')
ax.text(4.5, 3, 'user4\ncos=0.894', fontsize=10, va='center')
ax.text(4.5, 2, '← 類似ユーザーの\n   評価値を利用', fontsize=10, va='center', color='green')

ax.set_title('図2.27: user3のitem1の欠損値を、類似ユーザー（user1, user4）の評価から推計', fontsize=12)
plt.tight_layout()
plt.show()

## 予測値の計算式

user3のitem1に対する評価値 $r_{3,1}$ を計算するには、例えば次の数式に示すような方法があります。ちなみに評価値 $r_{3,1}$ は予測値として算出されるため、予測値を示す記号^（ハット）を用いて $\hat{r}_{3,1}$ と表記します。

$$\hat{r}_{3,1} = \bar{u}_3 + (r_{1,1} - \bar{u}_1) \times \cos(u_3, u_1) + (r_{4,1} - \bar{u}_4) \times \cos(u_3, u_4)$$

この数式は以下の構造になっています：

- **$\bar{u}_3$**：user3の平均評価値（ベース）
- **$(r_{1,1} - \bar{u}_1)$**：user1のitem1評価と平均との差
- **$\cos(u_3, u_1)$**：user3との類似度合い（重み付け）

In [ ]:
# 図2.28: 欠損値の推計で用いる数値を表形式で整理
print("=== 図2.28: 欠損値の推計で用いる数値 ===")
print()

# user1, user3, user4の情報
users_info = {
    'user1': {'ratings': R[0], 'item1': R[0, 0], 'mean': None, 'cos_with_u3': similarity_matrix[2, 0]},
    'user3': {'ratings': R[2], 'item1': '?', 'mean': None, 'cos_with_u3': '-'},
    'user4': {'ratings': R[3], 'item1': R[3, 0], 'mean': None, 'cos_with_u3': similarity_matrix[2, 3]}
}

# 平均値を計算（共通アイテムでの平均ではなく、各ユーザーの評価平均）
for user, info in users_info.items():
    ratings = info['ratings']
    valid_ratings = ratings[ratings != 0]
    info['mean'] = np.mean(valid_ratings)

print(f"{'':10} {'item1':>8} {'平均評価値':>12} {'user3とのcos':>14}")
print("-" * 50)
for user, info in users_info.items():
    item1_val = info['item1'] if isinstance(info['item1'], str) else f"{info['item1']}"
    cos_val = info['cos_with_u3'] if isinstance(info['cos_with_u3'], str) else f"{info['cos_with_u3']:.3f}"
    print(f"{user:10} {item1_val:>8} {info['mean']:>12.3f} {cos_val:>14}")

In [ ]:
# 予測値を計算
print("=== r̂_{3,1} の計算 ===")
print()

# 各値を取得
mean_u1 = users_info['user1']['mean']
mean_u3 = users_info['user3']['mean']
mean_u4 = users_info['user4']['mean']

r_1_1 = R[0, 0]  # user1のitem1評価 = 2
r_4_1 = R[3, 0]  # user4のitem1評価 = 4

cos_u3_u1 = similarity_matrix[2, 0]  # 0.614
cos_u3_u4 = similarity_matrix[2, 3]  # 0.894

print(f"各パラメータ:")
print(f"  ū₃ = {mean_u3:.3f}")
print(f"  r₁,₁ = {r_1_1}, ū₁ = {mean_u1:.3f}, (r₁,₁ - ū₁) = {r_1_1 - mean_u1:.3f}")
print(f"  r₄,₁ = {r_4_1}, ū₄ = {mean_u4:.3f}, (r₄,₁ - ū₄) = {r_4_1 - mean_u4:.3f}")
print(f"  cos(u₃, u₁) = {cos_u3_u1:.3f}")
print(f"  cos(u₃, u₄) = {cos_u3_u4:.3f}")
print()

# 予測値を計算
r_hat_3_1 = mean_u3 + (r_1_1 - mean_u1) * cos_u3_u1 + (r_4_1 - mean_u4) * cos_u3_u4

print(f"計算式:")
print(f"r̂₃,₁ = ū₃ + (r₁,₁ - ū₁) × cos(u₃, u₁) + (r₄,₁ - ū₄) × cos(u₃, u₄)")
print()
print(f"     = {mean_u3:.3f} + ({r_1_1 - mean_u1:.3f}) × {cos_u3_u1:.3f} + ({r_4_1 - mean_u4:.3f}) × {cos_u3_u4:.3f}")
print(f"     = {mean_u3:.3f} + {(r_1_1 - mean_u1) * cos_u3_u1:.3f} + {(r_4_1 - mean_u4) * cos_u3_u4:.3f}")
print(f"     = {r_hat_3_1:.3f}")
print()
print(f"予測結果: user3のitem1への評価値は {r_hat_3_1:.3f} と推計されました。")

In [ ]:
# 予測結果の解釈
print("=== 予測結果の解釈 ===")
print()
print(f"計算結果は {r_hat_3_1:.3f} と出ました。")
print()
print("user3はitem2に3、item3に4、item4に4の評価値を付与しているので、")
print(f"user3にとって {r_hat_3_1:.3f} という値は平均的な値とも考えられそうです。")
print()
print("であれば、そこまで積極的にitem1をレコメンドする必要はなさそうですし、")
print("レコメンドしたとしても購入される確率はそこまで高くなさそうだ、と推察されます。")
print()
print("ただし、この計算方法はあくまで一例であり、必ずしもこの通りに評価値を推定する")
print("必要はありません。どのように設計すれば目的にかなう数理モデルとなるか、常に")
print("考察と検証を繰り返すことが極めて重要です。")

---
# まとめ

## 本章で学んだこと

### 評価値行列
- ユーザーの評価をベクトルとして表現
- 複数ユーザーの評価を行列 $R$ としてまとめる
- 欠損値（0）= レコメンド対象

### 協調フィルタリング
- 類似ユーザーの評価を使って欠損値を予測
- 「類似性」をコサイン類似度で定量化

### コサイン類似度
$$\cos(a, b) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}||\vec{b}|} = \frac{\sum_{k=1}^{n} a_k b_k}{\sqrt{\sum_{k=1}^{n} a_k^2} \sqrt{\sum_{k=1}^{n} b_k^2}}$$

### 中心化（改良版）
$$\cos(a, b) = \frac{\sum_{k=1}^{n} (a_k - \bar{a})(b_k - \bar{b})}{\sqrt{\sum_{k=1}^{n} (a_k - \bar{a})^2} \sqrt{\sum_{k=1}^{n} (b_k - \bar{b})^2}}$$

### 指示関数
$$\delta_{u,i} = \begin{cases} 1 & (r_{u,i}\text{が評価値行列に含まれている}) \\ 0 & (r_{u,i}\text{が評価値行列で欠損している}) \end{cases}$$

### 欠損値の予測
$$\hat{r}_{3,1} = \bar{u}_3 + \sum_{u \in \text{similar users}} (r_{u,1} - \bar{u}) \times \cos(u_3, u)$$

In [ ]:
# 最終確認：学んだ関数のまとめ
print("=== 本Notebookで実装した関数 ===")
print()
print("1. cosine_similarity(a, b)")
print("   - 基本的なコサイン類似度を計算")
print()
print("2. cosine_similarity_centered(a, b)")
print("   - 中心化を行ったコサイン類似度を計算")
print("   - 欠損値（0）を自動的に除外")
print()
print("=== 使用例 ===")
print(f"基本版: cosine_similarity([2,3,5], [2,5,5]) = {cosine_similarity([2,3,5], [2,5,5]):.4f}")
print(f"中心化版: cosine_similarity_centered([2,3,0,5], [2,5,0,5]) = {cosine_similarity_centered([2,3,0,5], [2,5,0,5]):.4f}")

---
# 2-10 アイテム同士の類似度で予測値を推計する

前節まではuserを軸とした予測値の算出について解説しました。一方、類似度の計算方法としてはitem同士の類似度を用いることも可能です。

## userベースとitemベースの比較

| 手法 | 説明 |
|:---|:---|
| ①userを軸とした予測値の算出 | 似ているユーザーの評価値を利用 |
| ②itemを軸とした予測値の算出 | 似ているアイテムの評価値を利用 |

実際にECサイトで協調フィルタリングによって欠損値を推定する際は、**user軸よりもitem軸で予測値を算出することが実務上多い**ように見受けられます。

- ECサイトで扱うitem数と訪問してくるuser数を比較すると、user数のほうが多いことが通常
- user同士の類似度よりitem同士の類似度のほうが計算負荷が低くなりやすい

In [ ]:
# 図2.30: アイテム間の類似度行列
print("=== 図2.30: 各item同士の類似度は表形式で整理できる ===")
print()
print("          item1    item2    item3    item4")
print("item1   cos(i₁,i₁) cos(i₁,i₂) cos(i₁,i₃) cos(i₁,i₄)")
print("item2   cos(i₂,i₁) cos(i₂,i₂) cos(i₂,i₃) cos(i₂,i₄)")
print("item3   cos(i₃,i₁) cos(i₃,i₂) cos(i₃,i₃) cos(i₃,i₄)")
print("item4   cos(i₄,i₁) cos(i₄,i₂) cos(i₄,i₃) cos(i₄,i₄)")
print()
print("アイテムベースでは、評価値行列Rの「列ベクトル」を使って類似度を計算します。")

In [ ]:
# 評価値行列R（再掲）
R = np.array([
    [2, 3, 0, 5],
    [2, 5, 0, 5],
    [0, 3, 4, 4],
    [4, 2, 3, 0]
])

print("評価値行列 R:")
print(R)
print()

# 列ベクトルとして各アイテムを表現
i1 = R[:, 0]  # item1の列ベクトル
i2 = R[:, 1]  # item2の列ベクトル
i3 = R[:, 2]  # item3の列ベクトル
i4 = R[:, 3]  # item4の列ベクトル

print("各アイテムの列ベクトル:")
print(f"i₁ = {i1}")
print(f"i₂ = {i2}")
print(f"i₃ = {i3}")
print(f"i₄ = {i4}")

## アイテムベースのコサイン類似度関数

アイテムベースでも同様に、指示関数を用いて欠損値を除外し、中心化を行います。

$$\cos(i_1, i_4) = \frac{\sum_{u=1}^{4} \delta_{u,1}(r_{u,1} - \bar{u}_u) \delta_{u,4}(r_{u,4} - \bar{u}_u)}{\sqrt{\sum_{u=1}^{4} \delta_{u,1}(r_{u,1} - \bar{u}_u)^2} \sqrt{\sum_{u=1}^{4} \delta_{u,4}(r_{u,4} - \bar{u}_u)^2}}$$

In [ ]:
# アイテムベースのコサイン類似度関数
def cosine_similarity_item_based(R, item_i, item_j):
    """
    アイテムベースの中心化コサイン類似度を計算する
    
    Parameters:
    -----------
    R : numpy.ndarray
        評価値行列
    item_i, item_j : int
        比較するアイテムのインデックス
    
    Returns:
    --------
    float
        アイテム間のコサイン類似度
    """
    col_i = R[:, item_i]
    col_j = R[:, item_j]
    
    # 両方のアイテムを評価しているユーザーのみ
    common_users = (col_i != 0) & (col_j != 0)
    
    if np.sum(common_users) == 0:
        return 0.0
    
    # 各ユーザーの平均評価値で中心化
    numerator = 0.0
    norm_i_sq = 0.0
    norm_j_sq = 0.0
    
    for u in range(R.shape[0]):
        if common_users[u]:
            # ユーザーuの平均評価値（欠損値除く）
            user_ratings = R[u, :]
            valid_ratings = user_ratings[user_ratings != 0]
            mean_u = np.mean(valid_ratings)
            
            centered_i = col_i[u] - mean_u
            centered_j = col_j[u] - mean_u
            
            numerator += centered_i * centered_j
            norm_i_sq += centered_i ** 2
            norm_j_sq += centered_j ** 2
    
    if norm_i_sq == 0 or norm_j_sq == 0:
        return 0.0
    
    return numerator / (np.sqrt(norm_i_sq) * np.sqrt(norm_j_sq))

print("cosine_similarity_item_based関数を定義しました")

In [ ]:
# cos(i1, i4)を計算
print("=== cos(i₁, i₄) の計算 ===")
print()

# item1とitem4を共通で評価しているのはuser1とuser2
print("item1とitem4を共通で評価しているユーザー: user1, user2")
print("(user3はitem1が欠損、user4はitem4が欠損)")
print()

# user1とuser2の平均評価値
mean_u1 = np.mean([2, 3, 5])  # user1: [2,3,0,5] → 欠損除外
mean_u2 = np.mean([2, 5, 5])  # user2: [2,5,0,5] → 欠損除外
print(f"ū₁ = (2+3+5)/3 = {mean_u1:.3f}")
print(f"ū₂ = (2+5+5)/3 = {mean_u2:.3f}")
print()

# 中心化した値
print("中心化:")
print(f"  user1: (r₁,₁ - ū₁) = 2 - {mean_u1:.3f} = {2 - mean_u1:.3f}")
print(f"         (r₁,₄ - ū₁) = 5 - {mean_u1:.3f} = {5 - mean_u1:.3f}")
print(f"  user2: (r₂,₁ - ū₂) = 2 - {mean_u2:.3f} = {2 - mean_u2:.3f}")
print(f"         (r₂,₄ - ū₂) = 5 - {mean_u2:.3f} = {5 - mean_u2:.3f}")
print()

# コサイン類似度の計算
cos_i1_i4 = cosine_similarity_item_based(R, 0, 3)
print(f"cos(i₁, i₄) = {cos_i1_i4:.3f}")

In [ ]:
# 全アイテム間のコサイン類似度行列を計算
print("=== アイテム間の類似度行列 ===")
print()

n_items = R.shape[1]
item_similarity_matrix = np.zeros((n_items, n_items))

for i in range(n_items):
    for j in range(n_items):
        item_similarity_matrix[i, j] = cosine_similarity_item_based(R, i, j)

# 表形式で表示
print("          item1    item2    item3    item4")
for i, row in enumerate(item_similarity_matrix):
    print(f"item{i+1}  ", end="")
    for val in row:
        print(f"{val:8.3f} ", end="")
    print()

print()
print(f"cos(i₁, i₄) = {item_similarity_matrix[0, 3]:.3f}")
print(f"cos(i₂, i₄) = {item_similarity_matrix[1, 3]:.3f}")
print(f"cos(i₃, i₄) = {item_similarity_matrix[2, 3]:.3f}")
print()
print("→ item4に最も似ているのはitem3（cos=1.000）、次にitem2（cos=0.090）")

## アイテムベースでの欠損値予測

user4のitem4（$r_{4,4}$）の欠損値を、item軸で推計してみましょう。

item4に似ているのはitem2とitem3なので、user4のitem2, item3に対する評価値を参考にします。

In [ ]:
# 図2.31: 類似度の高いアイテムの評価値を用いて欠損値を推計
fig, ax = plt.subplots(figsize=(10, 6))

ax.imshow(np.where(R == 0, 0.3, 0.8), cmap='Blues', vmin=0, vmax=1)

for i in range(4):
    for j in range(4):
        if R[i, j] == 0:
            if i == 3 and j == 3:  # user4のitem4（推計対象）
                ax.text(j, i, '?', ha='center', va='center', fontsize=16, color='red', fontweight='bold')
            else:
                ax.text(j, i, '−', ha='center', va='center', fontsize=14, color='gray')
        else:
            ax.text(j, i, str(R[i, j]), ha='center', va='center', fontsize=14)

ax.set_xticks(range(4))
ax.set_yticks(range(4))
ax.set_xticklabels(['item1', 'item2', 'item3', 'item4'])
ax.set_yticklabels(['user1', 'user2', 'user3', 'user4'])

# 矢印で類似アイテムからの参照を示す
ax.annotate('', xy=(3, 3), xytext=(1, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))
ax.annotate('', xy=(3, 3), xytext=(2, 3),
            arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.set_title('図2.31: user4のitem4の欠損値を、類似アイテム（item2, item3）の評価から推計', fontsize=12)
plt.tight_layout()
plt.show()

print("user4の評価値: item1=4, item2=2, item3=3, item4=?")
print(f"cos(i₂, i₄) = {item_similarity_matrix[1, 3]:.3f}")
print(f"cos(i₃, i₄) = {item_similarity_matrix[2, 3]:.3f}")

---
# 2-11 ユーザー目線で数理モデルを再考する ーセレンディピティー

ここまでuser軸とitem軸の2つの観点から、評価値を予測するための数理モデルを解説してきました。ここで、今一度レコメンドの数理モデルについて考え直してみましょう。

## セレンディピティとは

ECサイトを考える上で重要な観点の1つに、**セレンディピティ**という概念があります。これは「目新しさ」といった意味合いで使われる概念です。

### item軸の問題点

item軸でのレコメンドを運用していると起きがちな現象：
- ECサイトに訪問するたびに毎回同じような商品ばかりがレコメンドされる
- ユーザーは見飽きてしまい、そのECサイトを利用しなくなることが懸念される
- 似ているアイテムばかりがレコメンドされても、そのユーザーが既に類似商品を持っていたりすると、レコメンドした商品への興味は薄い可能性が高い

In [ ]:
# セレンディピティの概念図
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左：item軸の問題点
ax1 = axes[0]
ax1.text(0.5, 0.9, 'item軸でのレコメンド', ha='center', fontsize=12, fontweight='bold', transform=ax1.transAxes)
ax1.text(0.5, 0.7, '毎回似たような商品ばかり', ha='center', fontsize=10, color='red', transform=ax1.transAxes)

# 同じような商品のイメージ
for i in range(3):
    ax1.add_patch(plt.Rectangle((0.2 + i*0.2, 0.3), 0.15, 0.2, fill=True, color='lightblue', alpha=0.8))
    ax1.text(0.275 + i*0.2, 0.4, f'商品{i+1}', ha='center', fontsize=9)

ax1.text(0.5, 0.15, '→ ユーザーは「また同じか...」と飽きてしまう', ha='center', fontsize=10, transform=ax1.transAxes)
ax1.axis('off')
ax1.set_title('問題：セレンディピティの欠如')

# 右：user軸の利点
ax2 = axes[1]
ax2.text(0.5, 0.9, 'user軸でのレコメンド', ha='center', fontsize=12, fontweight='bold', transform=ax2.transAxes)
ax2.text(0.5, 0.7, '自分と嗜好が似ているユーザーが\n興味を持っている商品を発見', ha='center', fontsize=10, color='green', transform=ax2.transAxes)

# 異なる商品のイメージ
colors = ['lightblue', 'lightgreen', 'lightyellow']
for i, color in enumerate(colors):
    ax2.add_patch(plt.Rectangle((0.2 + i*0.2, 0.3), 0.15, 0.2, fill=True, color=color, alpha=0.8))
    ax2.text(0.275 + i*0.2, 0.4, f'新発見{i+1}', ha='center', fontsize=9)

ax2.text(0.5, 0.15, '→ UX（User Experience）の向上が期待できる', ha='center', fontsize=10, transform=ax2.transAxes)
ax2.axis('off')
ax2.set_title('解決：意外な商品との出会い')

plt.tight_layout()
plt.show()

print("セレンディピティ = 思いがけない発見、偶然の出会い")
print()
print("user軸でのレコメンドは、自分と嗜好が似ている他のユーザーが興味を持っている")
print("アイテムに基づき欠損値を予測するため、item軸ではレコメンドされなかった")
print("商品に出会える可能性がある。")

## 協調フィルタリングの限界

つまり、item軸とuser軸、どちらのアプローチで欠損値を予測したとしても、セレンディピティという観点では優れたレコメンドを実現できない可能性がある、ということです。

では、セレンディピティという観点から考えると、どのような数理モデルによって評価値を予測すればよいのでしょうか。

この課題を解決するために、次節からより高度な数理モデルを考察します。

---
# 2-12 課題解決のために数理モデルを変更する ー行列因子分解ー

本節以降では、「③行列因子分解（MF: Matrix Factorization）」によって評価値を推定する数理モデルを導出します。その準備として、まずは**残差行列**を導出し、行列の**和**、**差**及び**積**への理解が必要となります。

## 基礎となる数理的手法

### 行列の和
$$A + B = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} + \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}+b_{1,1} & a_{1,2}+b_{1,2} \\ a_{2,1}+b_{2,1} & a_{2,2}+b_{2,2} \end{pmatrix}$$

### 行列の差
$$A - B = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} - \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}-b_{1,1} & a_{1,2}-b_{1,2} \\ a_{2,1}-b_{2,1} & a_{2,2}-b_{2,2} \end{pmatrix}$$

### 行列の積
$$AB = \begin{pmatrix} a_{1,1} & a_{1,2} \\ a_{2,1} & a_{2,2} \end{pmatrix} \begin{pmatrix} b_{1,1} & b_{1,2} \\ b_{2,1} & b_{2,2} \end{pmatrix} = \begin{pmatrix} a_{1,1}b_{1,1}+a_{1,2}b_{2,1} & a_{1,1}b_{1,2}+a_{1,2}b_{2,2} \\ a_{2,1}b_{1,1}+a_{2,2}b_{2,1} & a_{2,1}b_{1,2}+a_{2,2}b_{2,2} \end{pmatrix}$$

In [ ]:
# 行列演算の基礎
print("=== 行列演算の例 ===")
print()

A = np.array([[1, 2], [3, 1]])
B = np.array([[2, 3], [1, 2]])

print("行列A:")
print(A)
print()
print("行列B:")
print(B)
print()

print("--- 行列の和 A + B ---")
print(A + B)
print()

print("--- 行列の差 A - B ---")
print(A - B)
print()

print("--- 行列の積 AB ---")
print(A @ B)
print()
print("計算過程:")
print(f"  (1,1)要素: 1×2 + 2×1 = {1*2 + 2*1}")
print(f"  (1,2)要素: 1×3 + 2×2 = {1*3 + 2*2}")
print(f"  (2,1)要素: 3×2 + 1×1 = {3*2 + 1*1}")
print(f"  (2,2)要素: 3×3 + 1×2 = {3*3 + 1*2}")

## 行列積の規則性

行列同士の積の計算結果には規則性があります。重要なのは行列の**形**が重要だということです。

**規則**: $m \times n$ 行列と $n \times p$ 行列の積は $m \times p$ 行列になる

$$\underbrace{(2 \times 3)}_{A} \times \underbrace{(3 \times 2)}_{B} = \underbrace{(2 \times 2)}_{AB}$$

隣り合う数字が同じ（上の例では3）でないと、行列の積は計算できません。

In [ ]:
# 図2.36: 行列積が可能な場合と不可の場合
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# 左：計算可能
ax1 = axes[0]
ax1.text(0.1, 0.6, '4×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax1.text(0.25, 0.6, '×', fontsize=14, ha='center')
ax1.text(0.4, 0.6, '2×3', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightgreen'))
ax1.text(0.55, 0.6, '=', fontsize=14, ha='center')
ax1.text(0.7, 0.6, '4×3', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightyellow'))

# 矢印で一致を示す
ax1.annotate('', xy=(0.35, 0.45), xytext=(0.15, 0.45),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax1.text(0.25, 0.35, '一致!', fontsize=10, ha='center', color='green')

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')
ax1.set_title('計算可能', fontsize=12)

# 右：計算不可
ax2 = axes[1]
ax2.text(0.1, 0.6, '4×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue'))
ax2.text(0.25, 0.6, '×', fontsize=14, ha='center')
ax2.text(0.4, 0.6, '3×2', fontsize=14, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral'))
ax2.text(0.55, 0.6, '=', fontsize=14, ha='center')
ax2.text(0.7, 0.6, '×', fontsize=20, ha='center', color='red')

# 矢印で不一致を示す
ax2.annotate('', xy=(0.35, 0.45), xytext=(0.15, 0.45),
            arrowprops=dict(arrowstyle='<->', color='red', lw=2))
ax2.text(0.25, 0.35, '不一致!', fontsize=10, ha='center', color='red')

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')
ax2.set_title('計算できない', fontsize=12)

plt.suptitle('図2.36: 行列積が可能な場合と不可の場合', fontsize=12)
plt.tight_layout()
plt.show()

## 残差行列

評価値行列 $R$ と予測値行列 $\hat{R}$ の差を**残差行列** $E$ と呼びます。

$$E = R - \hat{R} = R - PQ$$

ここで：
- $R$: 評価値行列（$U \times I$ 行列）
- $\hat{R}$: 予測値行列（$U \times I$ 行列）
- $P$: ユーザー因子行列（$U \times d$ 行列）
- $Q$: アイテム因子行列（$d \times I$ 行列）
- $d$: **潜在因子**の数

## 潜在因子モデル

評価値行列がスパース（疎）な行列となりがちです。このような評価値行列が抱える実務上の問題に対応するために、数理モデルとして**潜在因子モデル**がしばしば活用されます。

潜在因子モデルでは、ユーザーとアイテムの関係を、いくつかの**潜在因子**に基づいて表現します。潜在因子とは、例えばユーザーの隠れた嗜好やアイテムの隠れた特性を反映しており、具体的な意味内容を持つこともあれば、データから抽出される抽象的な（意味付けることが難しい）特徴であることもあります。

## 行列因子分解（Matrix Factorization）

行列因子分解のアプローチでは、評価値行列 $R$ を**ユーザー因子行列** $P$ と**アイテム因子行列** $Q$ の積に分解します。

$$R \approx P \times Q$$

In [ ]:
# 図2.34: 行列因子分解の概念図
fig, ax = plt.subplots(figsize=(14, 5))

# R (U×I)
ax.add_patch(plt.Rectangle((0.05, 0.3), 0.15, 0.4, fill=True, color='lightblue', alpha=0.8))
ax.text(0.125, 0.5, 'R', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.125, 0.75, 'U×I', fontsize=10, ha='center')
ax.text(0.125, 0.2, '評価値行列', fontsize=9, ha='center')

# ≈
ax.text(0.25, 0.5, '≈', fontsize=24, ha='center', va='center')

# P (U×d)
ax.add_patch(plt.Rectangle((0.32, 0.3), 0.08, 0.4, fill=True, color='lightgreen', alpha=0.8))
ax.text(0.36, 0.5, 'P', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.36, 0.75, 'U×d', fontsize=10, ha='center')
ax.text(0.36, 0.2, 'ユーザー\n因子行列', fontsize=9, ha='center')

# ×
ax.text(0.45, 0.5, '×', fontsize=20, ha='center', va='center')

# Q (d×I)
ax.add_patch(plt.Rectangle((0.52, 0.4), 0.15, 0.15, fill=True, color='lightyellow', alpha=0.8))
ax.text(0.595, 0.475, 'Q', fontsize=20, ha='center', va='center', fontweight='bold')
ax.text(0.595, 0.6, 'd×I', fontsize=10, ha='center')
ax.text(0.595, 0.3, 'アイテム因子行列', fontsize=9, ha='center')

# 説明
ax.text(0.8, 0.6, 'd = 潜在因子の数', fontsize=11, ha='left')
ax.text(0.8, 0.5, 'U = ユーザー数', fontsize=11, ha='left')
ax.text(0.8, 0.4, 'I = アイテム数', fontsize=11, ha='left')

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.set_title('図2.34: 評価値行列Rをユーザー因子行列Pとアイテム因子行列Qに分解する', fontsize=12)
plt.tight_layout()
plt.show()

print("行列PとQは元の評価値行列Rよりも低ランクの行列となる。")
print("dは潜在因子の数で、ハイパーパラメータとして設定する。")

---
# 2-13 評価値の推計を最適化問題に置き換える ー残差行列と誤差ー

では、具体的な数理モデルの考察に入ります。このケースでもuser軸、item軸で考察した評価値行列を用います。

## 評価値行列の一般化表現

$$R = \begin{pmatrix} r_{1,1} & r_{1,2} & r_{1,3} & r_{1,4} \\ r_{2,1} & r_{2,2} & r_{2,3} & r_{2,4} \\ r_{3,1} & r_{3,2} & r_{3,3} & r_{3,4} \\ r_{4,1} & r_{4,2} & r_{4,3} & r_{4,4} \end{pmatrix} = \begin{pmatrix} 2 & 3 & 0 & 5 \\ 2 & 5 & 0 & 5 \\ 0 & 3 & 4 & 4 \\ 4 & 2 & 3 & 0 \end{pmatrix}$$

## 潜在因子行列の表現

$P$ は $4 \times 2$ 行列、$Q$ は $2 \times 4$ 行列です。

$$P = \begin{pmatrix} p_{1,1} & p_{1,2} \\ p_{2,1} & p_{2,2} \\ p_{3,1} & p_{3,2} \\ p_{4,1} & p_{4,2} \end{pmatrix}, \quad Q = \begin{pmatrix} q_{1,1} & q_{1,2} & q_{1,3} & q_{1,4} \\ q_{2,1} & q_{2,2} & q_{2,3} & q_{2,4} \end{pmatrix}$$

In [ ]:
# P, Qの一般化表現
print("=== 潜在因子行列の設定 ===")
print()

# d=2として設定
d = 2
n_users = 4
n_items = 4

print(f"潜在因子数 d = {d}")
print(f"ユーザー数 U = {n_users}")
print(f"アイテム数 I = {n_items}")
print()

print(f"ユーザー因子行列 P: {n_users}×{d} 行列")
print(f"アイテム因子行列 Q: {d}×{n_items} 行列")
print()

# 行列Pの要素
print("P の要素:")
for u in range(1, n_users+1):
    row = [f"p_{u},{k}" for k in range(1, d+1)]
    print(f"  user{u}: [{', '.join(row)}]")

print()

# 行列Qの要素
print("Q の要素:")
for k in range(1, d+1):
    row = [f"q_{k},{i}" for i in range(1, n_items+1)]
    print(f"  潜在因子{k}: [{', '.join(row)}]")

## 予測値の計算

$PQ$ の積を計算すると、各要素は以下のようになります。

$$\hat{r}_{u,i} = \sum_{k=1}^{d} p_{u,k} \cdot q_{k,i}$$

例えば $\hat{r}_{1,3}$（user1のitem3の予測値）は：

$$\hat{r}_{1,3} = p_{1,1} \cdot q_{1,3} + p_{1,2} \cdot q_{2,3}$$

## 残差（誤差）

評価値と予測値の差（残差）は：

$$e_{u,i} = r_{u,i} - \hat{r}_{u,i} = r_{u,i} - \sum_{k=1}^{d} p_{u,k} \cdot q_{k,i}$$

**行列因子分解の目標は、この残差を最小化するようなPとQを見つけること**です。

In [ ]:
# 行列因子分解の具体例
print("=== 行列因子分解の具体例 ===")
print()

# 評価値行列（5×4）
R_mf = np.array([
    [5, 3, 0, 1],
    [4, 0, 3, 2],
    [0, 5, 5, 5],
    [1, 2, 3, 4],
    [1, 0, 0, 5]
])

print("評価値行列 R (5ユーザー × 4アイテム):")
print(R_mf)
print()

# 仮にP, Qが解析的に求められたとする（d=2）
P = np.array([
    [0.3, 2.0],
    [0.7, 1.5],
    [1.9, 1.9],
    [1.6, 0.1],
    [2.0, 0.1]
])

Q = np.array([
    [0.4, 1.2, 1.7, 2.5],
    [2.4, 1.4, 1.0, 0.2]
])

print("ユーザー因子行列 P (5×2):")
print(P)
print()

print("アイテム因子行列 Q (2×4):")
print(Q)
print()

# PQを計算
PQ = P @ Q

print("予測行列 PQ (5×4):")
print(np.round(PQ, 2))
print()

print("元の評価値行列Rと予測行列PQを比較:")
print("R で既に評価値がついていた箇所と、PQ の計算結果は非常に近い値となっています。")
print("これが行列因子分解の大きな特徴です。")

In [ ]:
# アイテムベースでの予測値計算
print("=== r̂_{4,4} の計算（アイテムベース） ===")
print()

# user4の評価値
r_4_2 = R[3, 1]  # user4のitem2 = 2
r_4_3 = R[3, 2]  # user4のitem3 = 3

cos_i2_i4 = item_similarity_matrix[1, 3]  # 0.090
cos_i3_i4 = item_similarity_matrix[2, 3]  # 1.000

print("図2.32: 欠損値の推計で用いる数値")
print(f"  r₄,₂ = {r_4_2} (user4のitem2評価)")
print(f"  r₄,₃ = {r_4_3} (user4のitem3評価)")
print(f"  cos(i₂, i₄) = {cos_i2_i4:.3f}")
print(f"  cos(i₃, i₄) = {cos_i3_i4:.3f}")
print()

# 予測式: r̂_{4,4} = cos(i2,i4) × r_{4,2} + cos(i3,i4) × r_{4,3}
r_hat_4_4 = cos_i2_i4 * r_4_2 + cos_i3_i4 * r_4_3

print("計算式:")
print(f"r̂₄,₄ = cos(i₂, i₄) × r₄,₂ + cos(i₃, i₄) × r₄,₃")
print(f"     = {cos_i2_i4:.3f} × {r_4_2} + {cos_i3_i4:.3f} × {r_4_3}")
print(f"     = {cos_i2_i4 * r_4_2:.3f} + {cos_i3_i4 * r_4_3:.3f}")
print(f"     = {r_hat_4_4:.3f}")
print()
print(f"予測結果: user4のitem4への評価値は {r_hat_4_4:.3f} と推計されました。")
print("→ item4はuser4にとっては「可もなく不可もなく」という評価になりそうです。")